In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import yaml

# Load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Define paths
data_path = Path(config['output']['base_path'])
train_file = data_path / config['output']['train_file']
val_file = data_path / config['output']['val_file']
test_file = data_path / config['output']['test_file']
metadata_file = data_path / config['output']['metadata_file']
embeddings_file = data_path / config['output']['embeddings_file']

print("Checking file existence:")
for file in [train_file, val_file, test_file, metadata_file, embeddings_file]:
    print(f"✓ {file.name}" if file.exists() else f"✗ {file.name} (missing)")


Checking file existence:
✓ train_data.parquet
✓ val_data.parquet
✓ test_data.parquet
✓ metadata.parquet
✓ title_embeddings.parquet


In [2]:
# Load and verify data files
print("\nVerifying data files:")

# Load train data
train_df = pd.read_parquet(train_file)
print("\nTrain data:")
print(f"- Records: {len(train_df)}")
print(f"- Unique users: {train_df['user_id'].nunique()}")
print(f"- Unique items: {train_df['parent_asin'].nunique()}")
print("\nColumns:", train_df.columns.tolist())
print("\nSample data:")
display(train_df.head(2))

# Load validation data
val_df = pd.read_parquet(val_file)
print("\nValidation data:")
print(f"- Records: {len(val_df)}")
print(f"- Unique users: {val_df['user_id'].nunique()}")
print(f"- Unique items: {val_df['parent_asin'].nunique()}")

# Load test data
test_df = pd.read_parquet(test_file)
print("\nTest data:")
print(f"- Records: {len(test_df)}")
print(f"- Unique users: {test_df['user_id'].nunique()}")
print(f"- Unique items: {test_df['parent_asin'].nunique()}")

# Verify no overlap between splits
train_users = set(train_df['user_id'])
val_users = set(val_df['user_id'])
test_users = set(test_df['user_id'])

print("\nVerifying user overlap between splits:")
print(f"- Train-Val overlap: {len(train_users & val_users)} users")
print(f"- Train-Test overlap: {len(train_users & test_users)} users")
print(f"- Val-Test overlap: {len(val_users & test_users)} users")



Verifying data files:

Train data:
- Records: 2029
- Unique users: 253
- Unique items: 341

Columns: ['user_id', 'parent_asin', 'rating', 'timestamp', 'history']

Sample data:


,user_id,parent_asin,rating,timestamp,history
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07J3GH1W1,5.0,1547589356557,
1,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07W397QG4,5.0,1593352422858,B07J3GH1W1



Validation data:
- Records: 253
- Unique users: 253
- Unique items: 168

Test data:
- Records: 253
- Unique users: 253
- Unique items: 138

Verifying user overlap between splits:
- Train-Val overlap: 253 users
- Train-Test overlap: 253 users
- Val-Test overlap: 253 users


In [ ]:
# Verify metadata and embeddings
print("\nVerifying metadata and embeddings:")

# Load metadata
metadata_df = pd.read_parquet(metadata_file)
print("\nMetadata:")
print(f"- Records: {len(metadata_df)}")
print(f"- Unique parent ASINs: {metadata_df['parent_asin'].nunique()}")
print("\nColumns:", metadata_df.columns.tolist())
print("\nSample data:")
display(metadata_df.head(2))

# Load embeddings
embeddings_df = pd.read_parquet(embeddings_file)
print("\nEmbeddings:")
print(f"- Records: {len(embeddings_df)}")
print(f"- Unique parent ASINs: {embeddings_df['parent_asin'].nunique()}")
print(f"- Embedding dimension: {len(embeddings_df['embedding'].iloc[0])}")
print("\nColumns:", embeddings_df.columns.tolist())

# Verify coverage
all_parent_asins = set(pd.concat([train_df, val_df, test_df])['parent_asin'].unique())
metadata_asins = set(metadata_df['parent_asin'])
embeddings_asins = set(embeddings_df['parent_asin'])

print("\nVerifying coverage:")
print(f"- Total unique parent ASINs in data: {len(all_parent_asins)}")
print(f"- Parent ASINs with metadata: {len(metadata_asins)} ({len(metadata_asins & all_parent_asins)} in data)")
print(f"- Parent ASINs with embeddings: {len(embeddings_asins)} ({len(embeddings_asins & all_parent_asins)} in data)")

# Check for missing titles or embeddings
missing_metadata = all_parent_asins - metadata_asins
missing_embeddings = all_parent_asins - embeddings_asins

if missing_metadata:
    print("\nWarning: Missing metadata for ASINs:", list(missing_metadata)[:5], "...")
if missing_embeddings:
    print("\nWarning: Missing embeddings for ASINs:", list(missing_embeddings)[:5], "...")
